In [168]:
import django_initializer
from backend.pymongo.mongodb import db
from datetime import datetime
import arrow
from database.lss.campaign import Campaign
from backend.pymongo.mongodb import db
from channels.layers import get_channel_layer
from asgiref.sync import async_to_sync

In [218]:
after_15min = arrow.utcnow().replace(second=0, microsecond=0).shift(minutes=+15)
after_16min = after_15min.shift(minutes=+1)
after_1hour = arrow.utcnow().replace(second=0, microsecond=0).shift(minutes=+60)
after_61mins = after_1hour.shift(minutes=+1)
display(f"""
after_15min: {after_15min}
after_16min: {after_16min}
after_1hour: {after_1hour}
after_61mins: {after_61mins}
""")
# user_subscription = UserSubscription.objects.get(id=218)
# M_Campaign.objects.create(start_at=after_15min, end_at=after_1hour, user_subscription=user_subscription, title=F"test {after_16min}")
campaings_start_after_15mins = list(db.api_campaign.aggregate([
    {
        "$match":{
            "start_at":{
                "$gte": after_15min.datetime,
                "$lt": after_16min.datetime
            }
        }
    },
    {
        "$project":{
            "_id":0,
            "user_subscription_id":1,
            "title":1,
            "remind_time": "15 mins"
        }
    }
]))
campaings_start_after_1hour = list(db.api_campaign.aggregate([
    {
        "$match":{
            "start_at":{
                "$gte": after_1hour.datetime,
                "$lt": after_61mins.datetime
            }
        }
    },
    {
        "$project":{
            "_id":0,
            "user_subscription_id":1,
            "title":1,
            "remind_time": "an hour"
        }
    }
]))
campaigns = campaings_start_after_15mins + campaings_start_after_1hour
display(campaigns)
channel_layer = get_channel_layer()
display(dir(channel_layer))
for campaign in campaigns:
    user_subscription_id = campaign['user_subscription_id']
    title = campaign['title']
    remind_time = campaign['remind_time']
    display(f"user_subscription_id: {user_subscription_id}")
    display(f"title: {title}")
    display(f"remind_time: {remind_time}")
#     await channel_layer.group_send(f"user_subscription_{user_subscription_id}", {"type": "notification_message","data":{"message":{"title": title, "remind_time": remind_time}}})

'\nafter_15min: 2022-07-19T10:16:00+00:00\nafter_16min: 2022-07-19T10:17:00+00:00\nafter_1hour: 2022-07-19T11:01:00+00:00\nafter_61mins: 2022-07-19T11:02:00+00:00\n'

[{'user_subscription_id': 218,
  'title': 'test 2022-07-19 05:54:19.168968+00:00',
  'remind_time': '15 mins'}]

['ConnectionContextManager',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_backup_channel_name',
 '_brpop_with_clean',
 '_clean_receive_backup',
 '_group_key',
 '_map_channel_keys_to_connection',
 '_map_channel_to_connection',
 '_receive_index_generator',
 '_send_index_generator',
 '_setup_encryption',
 'brpop_timeout',
 'capacity',
 'channel_capacity',
 'channel_name_regex',
 'client_prefix',
 'close_pools',
 'compile_capacities',
 'connection',
 'consistent_hash',
 'crypter',
 'decode_hosts',
 'deserialize',
 'expiry',
 'extensions',
 'flush',
 'get_capacity',
 'group_add',
 'group_discard',
 'group_expiry',
 'group_name_regex',
 'group_send',
 'hosts',
 'invalid

'user_subscription_id: 218'

'title: test 2022-07-19 05:54:19.168968+00:00'

'remind_time: 15 mins'

In [208]:
from api.models.campaign.campaign import Campaign as M_Campaign
from api.models.user.user_subscription import UserSubscription
after_16min = arrow.utcnow().shift(minutes=+16).datetime
after_1hour = arrow.utcnow().shift(minutes=+60).datetime
user_subscription = UserSubscription.objects.get(id=218)
M_Campaign.objects.create(start_at=after_16min, end_at=after_1hour, user_subscription=user_subscription, title=F"test {after_16min}")

<Campaign: test 2022-07-19 09:24:52.818358+00:00>

In [192]:
from django_cron.management.commands.runcrons import run_cron_with_cache_check
from cron.cron import CampaignReminderCronJob
# run_cron_with_cache_check(CampaignReminderCronJob)

In [193]:
CampaignReminderCronJob

cron.cron.CampaignReminderCronJob